In [1]:
# %load StrategyLearner.py
"""
Template for implementing StrategyLearner  (c) 2016 Tucker Balch
"""
import numpy as np
import datetime as dt
import QLearner as ql
import pandas as pd
import util as ut
import time
class StrategyLearner(object):

    # constructor
    def __init__(self, verbose = False):
        self.verbose = verbose
        self.trained_pfl = None
        self.tested_pfl = None

    def get_feat(self, p, symbol):
        mmtn = pd.Series(index = p.index[1:], data = (p.values[1:]/p.values[:-1] - 1).flatten())
        accel = pd.Series(index = p.index[2:], data = mmtn[1:].values/mmtn[:-1].values - 1)

        sma20 = p.rolling(window = 20, min_periods = 0)
        sma20_mean = sma20.mean()
        sma20_std = sma20.std()
        bbup = sma20_mean + 2 * sma20_std
        bblow = sma20_mean - 2 * sma20_std
        bbvals = (p[1:] - sma20_mean[1:])/(4 * sma20_std[1:])
        vtl = sma20_std[1:]/sma20_mean[1:]
        sma20_mmtn = pd.Series(index = p.index[1:], data = (sma20_mean[1:].values/sma20_mean[:-1].values - 1).flatten())
        sma20_accel = pd.Series(index = p.index[2:], data = sma20_mmtn[1:].values/sma20_mmtn[:-1].values - 1)

        ema15 = p.ewm(span = 15, min_periods = 0)
        ema15_mean = ema15.mean()
        ema15_std = ema15.std()
        ema15_mmtn = pd.Series(index = p.index[1:], data = (ema15_mean[1:].values/ema15_mean[:-1].values - 1).flatten())
        ema15_accel = pd.Series(index = p.index[2:], data = ema15_mmtn[1:].values/ema15_mmtn[:-1].values - 1)
        ema40 = p.ewm(span = 40, min_periods = 0)
        ema40_mean = ema40.mean()
        ema40_std = ema40.std()
        ema40_mmtn = pd.Series(index = p.index[1:], data = (ema40_mean[1:].values/ema40_mean[:-1].values - 1).flatten())
        ema40_accel = pd.Series(index = p.index[2:], data = ema40_mmtn[1:].values/ema40_mmtn[:-1].values - 1)
        macd = ema15_mean - ema40_mean

        feats = pd.DataFrame({
            'price':p[symbol][2:],\
                    'mmtn':mmtn[1:],\
                    'accel':accel,\
                    'sma20_mean':sma20_mean[symbol][2:],\
                    'sma20_std':sma20_std[symbol][2:],\
                    'bbup':bbup[symbol][2:],\
                    'bblow':bblow[symbol][2:],\
                    'bbvals':bbvals[symbol][1:],\
                    'vtl':vtl[symbol][1:],\
                    'sma20_mmtn':sma20_mmtn[1:],\
                    'sma20_accel': sma20_accel,\
                    'ema15_mean': ema15_mean[symbol][2:],\
                    'ema15_std': ema15_std[symbol][2:],\
                    'ema15_mmtn': ema15_mmtn[1:],\
                    'ema15_accel': ema15_accel,\
                    'ema40_mean':ema40_mean[symbol][2:],\
                    'ema40_std':ema40_std[symbol][2:],\
                    'ema40_mmtn':ema40_mmtn[1:],\
                    'ema40_accel':ema40_accel,\
                    'macd':macd[symbol][2:]
                    })
        return feats

    def discretize(self, feats):
        x8 = (feats['mmtn'].values > 0).astype('int') # mmtn_sign 
        x7 = (np.abs(feats['mmtn'].values) >= 0.01).astype('int')#mmtn_mags  
        x6 = (feats['bbvals'].values > 0).astype('int') #bb_sign 
        x5 = (np.abs(feats['bbvals'].values) >= 0.5).astype('int') #bb_vals 
        x4 = (feats['macd'].values > 0).astype('int') #macd of today
        x3 = np.zeros(x4.size).astype('int') 
        x3[1:] = x4[:-1]#macd of yesterday
        x2 = (np.abs(feats['vtl'].values) >= 0.03).astype('int') #vtl 
        x1 = np.ones(x3.size).astype('int') # Holding: needs two bits to represnt: 0,-100; 1, 0;  2, +100.
        return pd.DataFrame(index = feats.index,\
                                    data = {'state': (x8<<8) + (x7<<7) + (x6<<6) +\
                                    (x5<<5) + (x4<<4) + (x3<<3) + (x2<<2) + x1})    


    def update_states(self, i, idx, action):
        """
        @param      i: current index(i.e. date)
        @param action: 0 -> nothing
                       1 -> buy 100
                       2 -> buy 200
                       3 -> sell 100
                       4 -> sell 200
        """
        holding = int(self.pfl.loc[idx[i],'state']) & 0b11 #holding: 0, -100; 1, 0; 2, 100 
        #print "+++++++++++++++++++++++++++++++++"
        #print "before action"
        #print "action: ", action, "  holding: ", holding, "  shares: ", self.pfl.loc[idx[i], 'shares']
        #print "pfl: ", self.pfl.loc[[idx[i]]]
        if action == 1:
            if holding <= 1:
                #print "case 1"
                self.pfl.loc[idx[i:], 'state'] += 1
                self.pfl.loc[idx[i:],'shares'] += 100
                self.pfl.loc[idx[i:],'cash'] -= 100*self.pfl.loc[idx[i],'price']
        elif action == 2:
            if holding == 0:
                #print "case 2-1"
                self.pfl.loc[idx[i:],'state'] += 2
                self.pfl.loc[idx[i:],'shares'] += 200
                self.pfl.loc[idx[i:],'cash'] -= 200*self.pfl.loc[idx[i],'price']
            elif holding == 1:
                #print "case 2-2"
                self.pfl.loc[idx[i:],'state'] += 1
                self.pfl.loc[idx[i:],'shares'] += 100
                self.pfl.loc[idx[i:],'cash'] -= 100*self.pfl.loc[idx[i],'price']
        elif action == 3:
            if holding >= 1:
                #print "case 3"
                self.pfl.loc[idx[i:],'state'] -= 1 
                self.pfl.loc[idx[i:],'shares'] -= 100
                self.pfl.loc[idx[i:],'cash'] += 100*self.pfl.loc[idx[i],'price']
        elif action == 4:
            if holding == 2:
                #print "case 4-1"
                self.pfl.loc[idx[i:],'state'] -= 2
                self.pfl.loc[idx[i:],'shares'] -= 200
                self.pfl.loc[idx[i:],'cash'] += 200*self.pfl.loc[idx[i],'price']
            elif holding == 1:
                #print "case 4-2"
                self.pfl.loc[idx[i:],'state'] -= 1 
                self.pfl.loc[idx[i:],'shares'] -= 100
                self.pfl.loc[idx[i:],'cash'] += 100*self.pfl.loc[idx[i],'price']

        self.pfl.loc[idx[i:],'portval'] = self.pfl.loc[idx[i:],'cash'] +\
                self.pfl.loc[idx[i:],'shares']*self.pfl.loc[idx[i:],'price']


    # this method should create a QLearner, and train it for trading
    def addEvidence(self, symbol = "IBM", \
            sd=dt.datetime(2008,1,1), \
            ed=dt.datetime(2009,1,1), \
            sv = 10000): 
        st = time.time() # for timing 
        self.ql= ql.QLearner(num_states = 511, num_actions = 5,rar = 0.9, radr = 0.999)
        # add your code to do learning here
        #symbol = 'ML4T-220' 
        # example usage of the old backward compatible util function
        syms=[symbol]
        dates = pd.date_range(sd, ed)
        prices_all = ut.get_data(syms, dates)  # automatically adds SPY
        prices = prices_all[syms]  # only portfolio symbols
        prices_SPY = prices_all['SPY']  # only SPY, for comparison later
        
        iterst = time.time()
        
        threshold = 0.001
        maxtime = 26
        dp = 1.0
        dt = 0
        itr = 0
        perfs = []
        while dp > threshold and dt < maxtime:
            feats = self.get_feat(prices, symbol)
            S_init = self.discretize(feats)  
            idx = feats.index
            self.pfl = pd.DataFrame(index = idx, \
                               data = {'price'  :prices_all[symbol][idx].values,\
                                       'shares' :np.zeros(len(idx)),\
                                       'cash'   :sv*np.ones(len(idx)),\
                                       'portval':np.zeros(len(idx)),\
                                       'state'  :S_init['state'].values})
            state = self.pfl['state'].ix[0]
            action = self.ql.querysetstate(state)
            self.update_states(0, idx, action)
            t = np.zeros(5)
            tt = np.zeros(4)
            for i in range(1, len(idx)):
                t[0]= time.time()
                r = (self.pfl.loc[idx[i],'portval']/self.pfl['portval'].ix[i-1] - 1)*100
                t[1] = time.time()
                state = int(self.pfl.loc[idx[i],'state'])
                t[2] = time.time()
                action = self.ql.query(state, r)
                t[3] = time.time()
                self.update_states(i, idx, action)
                t[4] = time.time()
                tt += np.diff(t)
            Np = self.pfl['portval'].ix[-1]/self.pfl['portval'].ix[0]
            if itr > 0:
                dp = abs(Np - perfs[-1])
            perfs.append(Np)
            itr += 1
            dt = time.time() - st
            #print "itr: ", itr, "  portval: ", Np , " tt[0]: ", tt[0]\
            #    , " tt[1]: ", tt[1]  , " tt[2]: ", tt[2]  , " tt[3]: ", tt[3]  
            
        #print "training time: ", time.time() - st, "  iteration time: ", time.time() - iterst     
        self.trained_pfl = self.pfl
        return self.pfl, self.ql

    # this method should use the existing policy and test it against new data
    def testPolicy(self, symbol = "IBM", \
            sd=dt.datetime(2009,1,1), \
            ed=dt.datetime(2010,1,1), \
            sv = 10000):

        # here we build a fake set of trades
        # your code should return the same sort of data
        #symbol = 'ML4T-220'
        dates = pd.date_range(sd, ed)
        prices_all = ut.get_data([symbol], dates)  # automatically adds SPY
        prices = prices_all[[symbol]]
        
        feats = self.get_feat(prices, symbol)
        S_init = self.discretize(feats)
        idx = feats.index
        self.pfl = pd.DataFrame(index = idx, \
                           data = {'price'  :prices_all[symbol][idx].values,\
                                   'shares' :np.zeros(len(idx)),\
                                   'cash'   :sv*np.ones(len(idx)),\
                                   'portval':np.zeros(len(idx)),\
                                   'state'  :S_init['state'].values})
        
        state = self.pfl['state'].ix[0]
        action = self.ql.querysetstate(state)
        self.update_states(0, idx, action)
        for i in range(1, len(idx)):
            r = (self.pfl.loc[idx[i],'portval']/self.pfl['portval'].ix[i-1] - 1)*100
            state = int(self.pfl.loc[idx[i],'state'])
            action = self.ql.query(state, r)
            self.update_states(i, idx, action)
        
        self.tested_pfl = self.pfl
        T = np.zeros(len(idx) + 3)
        T[3:] = self.pfl['shares'].values
        trades = pd.DataFrame(index = prices.index, data = {'trade': np.diff(T)})

        #trades = prices_all[[symbol,]]  # only portfolio symbols
        #trades_SPY = prices_all['SPY']  # only SPY, for comparison later
        #trades.values[:,:] = 0 # set them all to nothing
        #trades.values[3,:] = 100 # add a BUY at the 4th date
        #trades.values[5,:] = -100 # add a SELL at the 6th date 
        #trades.values[6,:] = -100 # add a SELL at the 7th date 
        #trades.values[8,:] = -100 # add a SELL at the 9th date
        #if self.verbose: print type(trades) # it better be a DataFrame!
        #if self.verbose: print trades
        #if self.verbose: print prices_all
        return trades

if __name__=="__main__":
    print "One does not simply think up a strategy, here we go"
    sl = StrategyLearner()
    pfl, ql = sl.addEvidence()
    pfl.to_csv('pfl.csv')
    ut.plot_data(pfl['portval']/pfl['portval'].ix[0])
    trades = sl.testPolicy()
    tpfl = sl.pfl
    tpfl.to_csv('tpfl.csv')
    ut.plot_data(tpfl['portval']/tpfl['portval'].ix[0])


One does not simply think up a strategy, here we go


In [4]:
pd.rolling_mean(pfl,window=20,min_periods=0)

/home/cshi31/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=0,window=20,center=False).mean()
  if __name__ == '__main__':


,cash,portval,price,shares,state
2008-01-04,713.000000,10000.000000,92.870000,100.000000,130.000000
2008-01-07,5307.000000,9950.500000,92.375000,50.000000,129.500000
2008-01-08,3851.000000,9934.000000,91.456667,66.666667,131.000000
2008-01-09,5380.000000,9942.250000,91.162500,50.000000,163.500000
2008-01-10,6297.400000,9947.200000,91.282000,40.000000,207.800000
2008-01-11,5414.000000,9950.500000,91.018333,50.000000,194.833333
2008-01-14,6133.428571,10021.857143,91.520000,42.857143,231.142857
2008-01-15,6673.000000,10075.375000,91.770000,37.500000,226.375000
2008-01-16,8129.666667,10117.000000,91.943333,22.222222,208.333333
2008-01-17,9295.000000,10155.100000,92.034000,10.000000,193.900000


(164, 4)

'0b10100100'